In [3]:
#!pip install flair
import psycopg2
import yaml
import pandas as pd
import re
from bs4 import BeautifulSoup
import time
from flair.data import Sentence
from flair.models import SequenceTagger

# Load configuration from YAML file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Database connection parameters
db_config = config['local_db']  # Adjust to 'remote_db' if needed

# Connect to the database
conn = psycopg2.connect(
    host=db_config['host'],
    port=db_config['port'],
    dbname=db_config['database'],
    user=db_config['user'],
    password=db_config['password']
)

# Define query parameters
query_limit = config.get('query_flair_limit', 2500000)
offset = config.get('offset', 0)
query_text = 'gold mine'

# SQL query to retrieve stories containing the keyword "gold mine"
story_query = f"SELECT id, story FROM public.\"DJ_NEWS_STORIES\" WHERE story LIKE '%{query_text}%' LIMIT {query_limit} OFFSET {offset};"

# Load Flair's pre-trained NER model
tagger = SequenceTagger.load("ner")

# Function to clean text: removes URLs, HTML tags, punctuation, numbers, and extra whitespace
def clean_text(text):
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Remove punctuation, digits, and non-printable characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Keep only letters and spaces
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace

    return text

# Start processing
start_time = time.time()

# Execute query to retrieve stories
cursor = conn.cursor()
cursor.execute(story_query)
result = cursor.fetchall()

# Collect entities and counts
unique_entities_set = set()
all_entities = set()
type_counts = {}
total_count = 0

# Process each story in the results
for row in result:
    story_id = row[0]
    text = row[1]
    
    # Clean the story text
    cleaned_text = clean_text(text)
    
    # Process cleaned text with Flair NER
    sentence = Sentence(cleaned_text)
    tagger.predict(sentence)
    
    # Extract and store relevant entities
    for entity in sentence.get_spans("ner"):
        if entity.get_label("ner").value in ['ORG', 'LOC', 'PER']:  # Use relevant entity types
            entity_type = entity.get_label("ner").value
            entity_tuple = (entity.text.strip(), entity_type, story_id)
            all_entities.add(entity_tuple)
            unique_entities_set.add((entity.text.strip(), entity_type))
            
            # Update counts for summary
            type_counts[entity_type] = type_counts.get(entity_type, 0) + 1
            total_count += 1

# Convert sets to sorted DataFrames for saving to Excel
unique_entities_df = pd.DataFrame(sorted(unique_entities_set), columns=["Entity", "Tag"])
all_entities_df = pd.DataFrame(sorted(all_entities), columns=["Entity", "Tag", "Story ID"])
summary_df = pd.DataFrame(list(type_counts.items()), columns=["Tag", "Count"])
summary_df = pd.concat([summary_df, pd.DataFrame([["TOTAL", total_count]], columns=["Tag", "Count"])])

# Save to Excel with multiple sheets
with pd.ExcelWriter("recognized_entities_flair.xlsx", engine="openpyxl") as writer:
    unique_entities_df.to_excel(writer, sheet_name="Unique Entities", index=False)
    all_entities_df.to_excel(writer, sheet_name="All Entities with Story ID", index=False)
    summary_df.to_excel(writer, sheet_name="Summary", index=False)

# Close the database connection
cursor.close()
conn.close()

# Print execution time
end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")


2024-10-27 14:35:55,213 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
Execution time: 14753.73 seconds


In [3]:
# https://flairnlp.github.io/docs/category/tutorial-2-training-models

  Using cached flair-0.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached boto3-1.35.49-py3-none-any.whl.metadata (6.7 kB)
  Using cached conllu-4.5.3-py2.py3-none-any.whl.metadata (19 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached huggingface_hub-0.26.1-py3-none-any.whl.metadata (13 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py) ... done
  Using cached more_itertools-10.5.0-py3-none-any.whl.metadata (36 kB)
  Using cached mpld3-0.5.10-py3-none-any.whl.metadata (5.1 kB)
  Using cached pptree-3.1.tar.gz (3.0 kB)
  Preparing metadata (setup.py) ... done
  Using cached pytorch_revgrad-0.2.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached segtok-1.5.11-py3-none-any.whl.metadata (9.0 kB)
  Using cached sqlitedict-2.1.0.tar.gz (21 kB)
  Preparing metadata (setup.py) ... done
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using 

/home/swiftx/anaconda3/envs/stanza_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-10-27 13:17:32,338 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
Sentence[4]: "I love Berlin ." → ["Berlin"/LOC]
